In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 브라우저 꺼짐 방지 옵션
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get("https://www.oliveyoung.co.kr/store/main/main.do")

time.sleep(2)

# 1. [카테고리] 버튼 클릭
driver.find_element(By.ID, "btnGnbOpen").click()
time.sleep(2)

# 2. '스킨케어' 링크 클릭 (텍스트 포함된 요소 찾기)
skin_care_link = driver.find_element(By.XPATH, "//a[contains(text(), '스킨케어')]")
skin_care_link.click()
time.sleep(3)

# 3. 첫 번째 상품 클릭
first_product = driver.find_element(By.CSS_SELECTOR, "ul.cate_prd_list > li:nth-child(1) .prd_info a")
first_product.click()
time.sleep(3)

# 4. 새 탭으로 전환 (상품 페이지가 새 창에서 열리는 경우 대비)
driver.switch_to.window(driver.window_handles[-1])

# 5. 리뷰 탭 클릭
try:
    review_tab = driver.find_element(By.CSS_SELECTOR, "#reviewInfo > a")
    review_tab.click()
    time.sleep(2)
except:
    print("리뷰 탭 없음")

# 6. 리뷰 내용 크롤링
reviews = driver.find_elements(By.CSS_SELECTOR, "#gdasList > li > div.review_cont > div.txt_inner")

print("\n[리뷰 내용]")
for i, review in enumerate(reviews, 1):
    print(f"{i}. {review.text.strip()}")

# 7. 페이지 넘기기
try:
    current_page = driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip()
    next_page_no = int(current_page) + 1

    next_page_button = driver.find_element(By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_page_no}"]')
    next_page_button.click()
    time.sleep(2)

    # 두 번째 페이지 리뷰 가져오기 (테스트용)
    print("\n[2페이지 리뷰 내용]")
    reviews = driver.find_elements(By.CSS_SELECTOR, "#gdasList > li > div.review_cont > div.txt_inner")
    for i, review in enumerate(reviews, 1):
        print(f"{i}. {review.text.strip()}")

except Exception as e:
    print(f"페이지 넘기기 실패: {e}")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"ul.cate_prd_list > li:nth-child(1) .prd_info a"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0106FC03+61635]
	GetHandleVerifier [0x0106FC44+61700]
	(No symbol) [0x00E905D3]
	(No symbol) [0x00ED899E]
	(No symbol) [0x00ED8D3B]
	(No symbol) [0x00F20E12]
	(No symbol) [0x00EFD2E4]
	(No symbol) [0x00F1E61B]
	(No symbol) [0x00EFD096]
	(No symbol) [0x00ECC840]
	(No symbol) [0x00ECD6A4]
	GetHandleVerifier [0x012F4523+2701795]
	GetHandleVerifier [0x012EFCA6+2683238]
	GetHandleVerifier [0x0130A9EE+2793134]
	GetHandleVerifier [0x010868C5+155013]
	GetHandleVerifier [0x0108CFAD+181357]
	GetHandleVerifier [0x01077458+92440]
	GetHandleVerifier [0x01077600+92864]
	GetHandleVerifier [0x01061FF0+5296]
	BaseThreadInitThun

In [3]:
# ─────────────────────────────────────────
# 필요한 패키지 설치 (최초 1회만)
# pip install selenium webdriver-manager pandas openpyxl
# ─────────────────────────────────────────

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

def go_to_next_page(driver):
    """
    현재 페이지(+1)로 이동.
    같은 10개 그룹 내에 다음 페이지가 없으면 '다음 10 페이지' 버튼을 클릭.
    """
    curr = int(driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip())
    next_no = curr + 1
    try:
        # 같은 그룹 내에서 단순 페이지 번호 클릭
        btn = driver.find_element(By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')
        btn.click()
    except NoSuchElementException:
        # 같은 그룹에 없다면 '다음 10 페이지' 클릭 후 재시도
        nxt10 = driver.find_element(By.CSS_SELECTOR, "div.pageing > a.next")
        nxt10.click()
        time.sleep(2)
        btn = driver.find_element(By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')
        btn.click()
    time.sleep(2)

def main():
    # 1) 드라이버 실행 (크롬 자동 업데이트 + 창 유지 옵션)
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    # 2) 올리브영 메인 페이지 접속
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")
    time.sleep(2)

    # 3) 카테고리 탭 열기
    driver.find_element(By.ID, "btnGnbOpen").click()
    time.sleep(1)

    # 4) '스킨케어' 대분류 클릭
    driver.find_element(By.XPATH, "//a[contains(text(), '스킨케어')]").click()
    time.sleep(2)

    # 5) 첫 번째 상품 클릭 → 새 탭 전환
    driver.find_element(By.CSS_SELECTOR, "ul.cate_prd_list > li:nth-child(1) .prd_info a").click()
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)

    # 6) 리뷰 탭 클릭 (없으면 종료)
    try:
        driver.find_element(By.CSS_SELECTOR, "#reviewInfo > a").click()
        time.sleep(2)
    except NoSuchElementException:
        print("❌ 리뷰 탭이 없습니다.")
        driver.quit()
        return

    # 7) 리뷰 수집 루프
    all_reviews = []
    while True:
        # 7-1) 현재 페이지 리뷰 수집
        reviews = driver.find_elements(By.CSS_SELECTOR, "#gdasList > li > div.review_cont > div.txt_inner")
        for r in reviews:
            txt = r.text.strip()
            if txt:
                all_reviews.append(txt)

        # 7-2) 다음 페이지로 이동
        try:
            go_to_next_page(driver)
        except NoSuchElementException:
            # 더 이상 넘길 페이지가 없으면 루프 종료
            break

    # 8) DataFrame 생성 및 파일 저장
    df = pd.DataFrame(all_reviews, columns=["리뷰내용"])
    df.to_csv("oliveyoung_reviews.csv", index=False, encoding="utf-8-sig")
    df.to_excel("oliveyoung_reviews.xlsx", index=False)

    print(f"✅ 총 {len(all_reviews)}개의 리뷰를 수집했습니다.")
    print("✅ 파일 저장 완료: oliveyoung_reviews.csv, oliveyoung_reviews.xlsx")

    driver.quit()

if __name__ == "__main__":
    main()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"ul.cate_prd_list > li:nth-child(1) .prd_info a"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0106FC03+61635]
	GetHandleVerifier [0x0106FC44+61700]
	(No symbol) [0x00E905D3]
	(No symbol) [0x00ED899E]
	(No symbol) [0x00ED8D3B]
	(No symbol) [0x00F20E12]
	(No symbol) [0x00EFD2E4]
	(No symbol) [0x00F1E61B]
	(No symbol) [0x00EFD096]
	(No symbol) [0x00ECC840]
	(No symbol) [0x00ECD6A4]
	GetHandleVerifier [0x012F4523+2701795]
	GetHandleVerifier [0x012EFCA6+2683238]
	GetHandleVerifier [0x0130A9EE+2793134]
	GetHandleVerifier [0x010868C5+155013]
	GetHandleVerifier [0x0108CFAD+181357]
	GetHandleVerifier [0x01077458+92440]
	GetHandleVerifier [0x01077600+92864]
	GetHandleVerifier [0x01061FF0+5296]
	BaseThreadInitThun

In [5]:
# ─────────────────────────────────────────
# 필요한 패키지 설치 (최초 1회만)
# pip install selenium webdriver-manager pandas openpyxl
# ─────────────────────────────────────────

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

def go_to_next_page(driver, wait):
    """현재 페이지+1로 이동. 같은 10개 그룹 내에 다음 페이지가 없으면 '다음 10 페이지' 클릭."""
    curr = int(driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip())
    next_no = curr + 1
    try:
        btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')))
        btn.click()
    except TimeoutException:
        # 같은 그룹에 해당 번호가 없으면 '다음 10 페이지' 클릭
        nxt10 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pageing > a.next")))
        nxt10.click()
        # 그룹 전환 후, 다시 next_no 클릭
        btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')))
        btn.click()
    # 페이지가 바뀔 때까지 잠시 기다림
    wait.until(lambda d: int(d.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip()) == next_no)

def main():
    # 1) 크롬 드라이버 셋업
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    wait = WebDriverWait(driver, 10)

    # 2) 올리브영 메인 페이지 접속
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")

    # 3) 카테고리 열기
    wait.until(EC.element_to_be_clickable((By.ID, "btnGnbOpen"))).click()

    # 4) '스킨케어' 대분류 클릭
    wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '스킨케어')]"))).click()

    # 5) 첫 번째 상품 클릭 — 링크가 새 탭/창으로 열릴 수도 있으므로 핸들 관리
    prod_link = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "ul.cate_prd_list > li:nth-child(1) .prd_info a")))
    original_handles = driver.window_handles.copy()
    prod_link.click()

    # 6) 새 탭이 열렸다면 전환, 아니면 그대로
    wait.until(lambda d: len(d.window_handles) >= len(original_handles))
    new_handles = driver.window_handles
    if len(new_handles) > len(original_handles):
        # 새로 열린 탭으로 전환
        new_tab = [h for h in new_handles if h not in original_handles][0]
        driver.switch_to.window(new_tab)

    # 7) 리뷰 탭 클릭 (없으면 종료)
    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#reviewInfo > a"))).click()
    except TimeoutException:
        print("❌ 이 상품에는 리뷰 탭이 없습니다.")
        driver.quit()
        return

    # 8) 모든 페이지 리뷰 수집
    all_reviews = []
    while True:
        # 8-1) 현재 페이지 리뷰 수집
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#gdasList > li")))
        items = driver.find_elements(By.CSS_SELECTOR, "#gdasList > li > div.review_cont > div.txt_inner")
        for it in items:
            txt = it.text.strip()
            if txt:
                all_reviews.append(txt)

        # 8-2) 다음 페이지로 이동
        try:
            go_to_next_page(driver, wait)
        except (TimeoutException, NoSuchElementException):
            break

    # 9) DataFrame 생성 및 파일로 저장
    df = pd.DataFrame(all_reviews, columns=["리뷰내용"])
    df.to_csv("oliveyoung_reviews.csv", index=False, encoding="utf-8-sig")
    df.to_excel("oliveyoung_reviews.xlsx", index=False)

    print(f"✅ 총 {len(all_reviews)}개의 리뷰를 수집했습니다.")
    print("✅ 저장 완료: oliveyoung_reviews.csv, oliveyoung_reviews.xlsx")

    driver.quit()

if __name__ == "__main__":
    main()


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0106FC03+61635]
	GetHandleVerifier [0x0106FC44+61700]
	(No symbol) [0x00E905D3]
	(No symbol) [0x00ED899E]
	(No symbol) [0x00ED8D3B]
	(No symbol) [0x00F20E12]
	(No symbol) [0x00EFD2E4]
	(No symbol) [0x00F1E61B]
	(No symbol) [0x00EFD096]
	(No symbol) [0x00ECC840]
	(No symbol) [0x00ECD6A4]
	GetHandleVerifier [0x012F4523+2701795]
	GetHandleVerifier [0x012EFCA6+2683238]
	GetHandleVerifier [0x0130A9EE+2793134]
	GetHandleVerifier [0x010868C5+155013]
	GetHandleVerifier [0x0108CFAD+181357]
	GetHandleVerifier [0x01077458+92440]
	GetHandleVerifier [0x01077600+92864]
	GetHandleVerifier [0x01061FF0+5296]
	BaseThreadInitThunk [0x75CE00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77E67BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77E67B8E+238]


In [10]:
# ─────────────────────────────────────────
# 필요한 패키지 설치 (최초 1회만)
# pip install selenium webdriver-manager pandas openpyxl
# ─────────────────────────────────────────

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

def go_to_next_page(driver, wait):
    """
    현재 페이지 번호를 읽어 올리지 못하거나, 비어 있으면
    더 이상 페이지가 없다고 보고 NoSuchElementException 발생.
    """
    # 1) 현재 페이지 번호 안전하게 파싱
    try:
        curr_str = driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip()
        curr = int(curr_str)
    except (NoSuchElementException, ValueError):
        # 페이지 정보가 없거나 비어 있으면 종료 신호
        raise NoSuchElementException("페이지 번호를 파싱할 수 없습니다.")

    next_no = curr + 1

    # 2) 같은 10개 그룹 내에서 단순 페이지 이동
    try:
        btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')))
        btn.click()

    # 3) 같은 그룹에 없으면 '다음 10 페이지' 버튼 클릭 후 재시도
    except TimeoutException:
        nxt10 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pageing > a.next")))
        nxt10.click()
        # 그룹 전환 후, 다시 next_no 번호 클릭
        btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')))
        btn.click()

    # 4) 페이지가 실제로 바뀔 때까지 대기
    wait.until(lambda d: d.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip() == str(next_no))


def main():
    # 1) 크롬 드라이버 셋업
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)  # 창이 안 닫히도록
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    wait = WebDriverWait(driver, 10)

    # 2) 올리브영 메인 페이지 접속
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")

    # 3) 카테고리 열기
    wait.until(EC.element_to_be_clickable((By.ID, "btnGnbOpen"))).click()

    # 4) '스킨/토너' 카테고리 클릭
    wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '스킨/토너')]"))).click()

    # 5) 첫 번째 상품 클릭 (새 탭/창 핸들 관리)
    prod_link = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "ul.cate_prd_list > li:nth-child(1) .prd_info a")))
    original_handles = driver.window_handles.copy()
    prod_link.click()

    # 6) 새 탭이 열렸으면 전환
    wait.until(lambda d: len(d.window_handles) >= len(original_handles))
    if len(driver.window_handles) > len(original_handles):
        new_tab = [h for h in driver.window_handles if h not in original_handles][0]
        driver.switch_to.window(new_tab)

    # 7) 리뷰 탭 클릭 (없으면 스크립트 종료)
    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#reviewInfo > a"))).click()
    except TimeoutException:
        print("❌ 이 상품에는 리뷰 탭이 없습니다.")
        driver.quit()
        return

    # 8) 모든 페이지 리뷰 수집
    all_reviews = []
    while True:
        # 8-1) 현재 페이지 리뷰 수집
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#gdasList > li")))
        for elem in driver.find_elements(By.CSS_SELECTOR, "#gdasList > li > div.review_cont > div.txt_inner"):
            txt = elem.text.strip()
            if txt:
                all_reviews.append(txt)

        # 8-2) 다음 페이지로 이동 시도
        try:
            go_to_next_page(driver, wait)
        except NoSuchElementException:
            # 더 이상 이동할 페이지가 없으면 루프 탈출
            break

    # 9) DataFrame 생성 및 CSV/Excel 저장
    df = pd.DataFrame(all_reviews, columns=["리뷰내용"])
    df.to_csv("oliveyoung_reviews.csv", index=False, encoding="utf-8-sig")
    df.to_excel("oliveyoung_reviews.xlsx", index=False)

    print(f"✅ 총 {len(all_reviews)}개의 리뷰를 수집했습니다.")
    print("✅ 파일 저장 완료: oliveyoung_reviews.csv, oliveyoung_reviews.xlsx")

    driver.quit()


if __name__ == "__main__":
    main()


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0106FC03+61635]
	GetHandleVerifier [0x0106FC44+61700]
	(No symbol) [0x00E905D3]
	(No symbol) [0x00ED899E]
	(No symbol) [0x00ED8D3B]
	(No symbol) [0x00F20E12]
	(No symbol) [0x00EFD2E4]
	(No symbol) [0x00F1E61B]
	(No symbol) [0x00EFD096]
	(No symbol) [0x00ECC840]
	(No symbol) [0x00ECD6A4]
	GetHandleVerifier [0x012F4523+2701795]
	GetHandleVerifier [0x012EFCA6+2683238]
	GetHandleVerifier [0x0130A9EE+2793134]
	GetHandleVerifier [0x010868C5+155013]
	GetHandleVerifier [0x0108CFAD+181357]
	GetHandleVerifier [0x01077458+92440]
	GetHandleVerifier [0x01077600+92864]
	GetHandleVerifier [0x01061FF0+5296]
	BaseThreadInitThunk [0x75CE00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77E67BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77E67B8E+238]


In [ ]:
# 위에꺼는 잘되는것 

In [ ]:
# ─────────────────────────────────────────
# 필요 패키지 (최초 1회만)
# pip install selenium webdriver-manager pandas openpyxl
# ─────────────────────────────────────────

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

def go_to_next_page(driver, wait):
    """
    현재 페이지 번호(curr)를 읽어, curr+1 페이지로 이동.
    같은 10페이지 그룹 내에 next_no가 없으면 '다음 10 페이지' 버튼을 눌러 건너뛴 뒤 클릭.
    이동 성공 시 True, 더 이상 없으면 False 반환.
    """
    try:
        curr_str = driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip()
        curr = int(curr_str)
    except (NoSuchElementException, ValueError):
        return False

    next_no = curr + 1

    # 1) 같은 그룹 내 번호 클릭 시도
    try:
        btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')))
        btn.click()
    except TimeoutException:
        # 2) 그룹 내 없으면 '다음 10 페이지' 클릭
        try:
            nxt10 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pageing > a.next")))
            nxt10.click()
            # 3) 건너뛴 그룹에서 번호 클릭
            btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')))
            btn.click()
        except TimeoutException:
            return False

    # 4) 실제 페이지가 next_no가 될 때까지 대기
    try:
        wait.until(lambda d: d.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip() == str(next_no))
        return True
    except TimeoutException:
        return False

def collect_reviews(driver, wait):
    """현재 상품 리뷰 탭에서 1페이지부터 끝까지 텍스트를 리턴하는 함수."""
    reviews = []

    # 리뷰 탭이 로드될 때까지
    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#reviewInfo > a"))).click()
    except TimeoutException:
        return reviews

    # 페이지 순회
    while True:
        # 리뷰 아이템 대기 및 수집
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#gdasList > li")))
        elems = driver.find_elements(By.CSS_SELECTOR, "#gdasList > li > div.review_cont > div.txt_inner")
        for e in elems:
            txt = e.text.strip()
            if txt:
                reviews.append(txt)

        # 다음 페이지 이동, 없으면 종료
        if not go_to_next_page(driver, wait):
            break

    return reviews

def main():
    # 1) 드라이버 & 대기 객체 셋업
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    wait = WebDriverWait(driver, 10)

    # 2) 올리브영 메인 접속
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")

    # 3) 카테고리 메뉴 열기
    wait.until(EC.element_to_be_clickable((By.ID, "btnGnbOpen"))).click()

    # 수집할 카테고리 리스트
    categories = ["스킨/토너", "에센스/세럼/앰플"]

    all_reviews = []

    for idx, cat in enumerate(categories):
        # ─────────────────── 카테고리 클릭 ───────────────────
        if idx == 0:
            # 첫 카테고리는 대메뉴에서 바로 클릭
            wait.until(EC.element_to_be_clickable((By.XPATH, f"//a[contains(text(), '{cat}')]"))).click()
        else:
            # 다음 카테고리는 브레드크럼(loc_history)에서 클릭
            wait.until(EC.element_to_be_clickable((
                By.XPATH,
                f"//ul[@class='loc_history']//a[@class='cate_y' and normalize-space(text())='{cat}']"
            ))).click()

        # 4) 상품 리스트 로드 대기
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.cate_prd_list > li .prd_info a")))

        # 5) 현재 창 핸들 저장
        main_handle = driver.current_window_handle

        # 6) 상품 링크(절대 URL) 수집
        links = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li .prd_info a")
        hrefs = [a.get_attribute("href") for a in links]

        # 7) 각 상품별로 새로운 탭 열어 리뷰 수집
        for href in hrefs:
            driver.execute_script("window.open(arguments[0], '_blank');", href)
            # 새 탭으로 전환
            driver.switch_to.window(driver.window_handles[-1])

            # 리뷰 수집
            reviews = collect_reviews(driver, wait)
            all_reviews.extend(reviews)

            # 탭 닫고 원래로 복귀
            driver.close()
            driver.switch_to.window(main_handle)

    # 8) 결과 저장
    df = pd.DataFrame(all_reviews, columns=["리뷰내용"])
    df.to_csv("oliveyoung_reviews.csv", index=False, encoding="utf-8-sig")
    df.to_excel("oliveyoung_reviews.xlsx", index=False)

    print(f"✅ 총 {len(all_reviews)}개의 리뷰를 수집했습니다.")
    print("✅ 저장 완료: oliveyoung_reviews.csv, oliveyoung_reviews.xlsx")

    driver.quit()

if __name__ == "__main__":
    main()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# --- 이전에 정의한 함수들 ---
def go_to_next_page(driver, wait):
    try:
        curr = int(driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip())
    except:
        return False
    nxt = curr + 1
    # 같은 그룹 내 페이지 번호 클릭
    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{nxt}"]'))).click()
    except TimeoutException:
        # 없으면 다음 10페이지 클릭 후 재시도
        try:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pageing > a.next"))).click()
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{nxt}"]'))).click()
        except TimeoutException:
            return False
    # 실제로 페이지가 바뀔 때까지 기다림
    try:
        wait.until(lambda d: d.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip() == str(nxt))
        return True
    except:
        return False

def collect_reviews(driver, wait):
    reviews = []
    # 리뷰 탭 클릭
    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#reviewInfo > a"))).click()
        time.sleep(1)
    except TimeoutException:
        return reviews

    # 페이지 순회
    while True:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#gdasList > li")))
        for el in driver.find_elements(By.CSS_SELECTOR, "#gdasList > li > div.review_cont > div.txt_inner"):
            txt = el.text.strip()
            if txt:
                reviews.append(txt)
        if not go_to_next_page(driver, wait):
            break

    return reviews

# --- 여기부터 main 스크립트 ---
def main():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 10)

    # 1) 올리브영 스킨케어 카테고리 첫 화면 진입 (이전 코드 그대로)
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")
    wait.until(EC.element_to_be_clickable((By.ID, "btnGnbOpen"))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '스킨/토너')]"))).click()
    # (필요하다면 에센스/세럼 클릭 코드도 여기에)

    # --- A. “크림” 브레드크럼 클릭 & 리뷰 수집 ---
    wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//ul[@class='loc_history']//a[@class='cate_y' and normalize-space(text())='크림']"
    ))).click()
    time.sleep(1)

    cream_reviews = collect_reviews(driver, wait)
    pd.DataFrame(cream_reviews, columns=["리뷰내용"]) \
      .to_csv("cream_reviews.csv", index=False, encoding="utf-8-sig")
    print(f"✅ 크림 리뷰 {len(cream_reviews)}개 저장됨 → cream_reviews.csv")

    # --- B. “아이크림” 서브카테고리 클릭 & 리뷰 수집 ---
    wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//ul[@class='cate_list_box']//a[normalize-space(text())='아이크림']"
    ))).click()
    time.sleep(1)

    eye_reviews = collect_reviews(driver, wait)
    pd.DataFrame(eye_reviews, columns=["리뷰내용"]) \
      .to_csv("eyecream_reviews.csv", index=False, encoding="utf-8-sig")
    print(f"✅ 아이크림 리뷰 {len(eye_reviews)}개 저장됨 → eyecream_reviews.csv")

    driver.quit()

if __name__ == "__main__":
    main()


In [ ]:
# ─────────────────────────────────────────
# 필요 패키지 (최초 1회만)
# pip install selenium webdriver-manager pandas openpyxl
# ─────────────────────────────────────────

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.remote.remote_connection import RemoteConnection
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    WebDriverException,
)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# ─── 0) HTTP 통신 타임아웃을 300초로 연장 ───
RemoteConnection.set_timeout(300)

def go_to_next_page(driver, wait):
    try:
        curr_str = driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip()
        if not curr_str.isdigit():
            return False
        curr = int(curr_str)
    except (NoSuchElementException, WebDriverException):
        return False

    next_no = curr + 1

    # 같은 그룹 내 번호 클릭 시도
    try:
        btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')))
        btn.click()
    except TimeoutException:
        # '다음 10페이지' 그룹 클릭 후 다시 시도
        try:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pageing > a.next"))).click()
            wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]'))).click()
        except TimeoutException:
            return False

    # 실제 페이지 전환 확인
    try:
        wait.until(
            lambda d: d.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip() == str(next_no)
        )
        return True
    except TimeoutException:
        return False


def collect_reviews(driver, wait):
    reviews = []
    # 리뷰 탭 열기
    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#reviewInfo > a"))).click()
    except TimeoutException:
        return reviews

    while True:
        try:
            # 짧게 5초만 대기
            short_wait = WebDriverWait(driver, 5, poll_frequency=0.5)
            short_wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#gdasList > li")))

            elems = driver.find_elements(
                By.CSS_SELECTOR,
                "#gdasList > li > div.review_cont > div.txt_inner"
            )
            for e in elems:
                txt = e.text.strip()
                if txt:
                    reviews.append(txt)

        except TimeoutException:
            break
        except WebDriverException:
            break

        if not go_to_next_page(driver, wait):
            break

    return reviews


def main():
    # ─── 1) Chrome 옵션 세팅 ───────────────────
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "profile.managed_default_content_settings.images": 2
    })
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_experimental_option("detach", True)

    # ─── 2) 드라이버 및 타임아웃 설정 ───────────
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    driver.set_page_load_timeout(60)
    driver.set_script_timeout(30)
    wait = WebDriverWait(driver, 10, poll_frequency=0.5)

    # ─── 3) 메인 접속 ───────────────────────────
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")

    categories = ["스킨/토너", "에센스/세럼/앰플"]
    all_reviews = []

    for cat in categories:
        # GNB 메뉴 열고 카테고리 클릭
        wait.until(EC.element_to_be_clickable((By.ID, "btnGnbOpen"))).click()
        wait.until(EC.element_to_be_clickable((By.LINK_TEXT, cat))).click()

        # 상품 리스트 로딩 대기
        wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "ul.cate_prd_list > li .prd_info a")
        ))
        main_handle = driver.current_window_handle

        # 상품 링크 수집
        hrefs = [
            a.get_attribute("href")
            for a in driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li .prd_info a")
        ]

        for href in hrefs:
            try:
                driver.execute_script("window.open(arguments[0], '_blank');", href)
                driver.switch_to.window(driver.window_handles[-1])

                reviews = collect_reviews(driver, wait)
                all_reviews.extend(reviews)

            except Exception as e:
                print(f"❗️ 리뷰 수집 중 오류: {e}")

            finally:
                driver.close()
                driver.switch_to.window(main_handle)

    # ─── 4) 결과 저장 ───────────────────────────
    df = pd.DataFrame(all_reviews, columns=["리뷰내용"])
    df.to_csv("oliveyoung_reviews.csv", index=False, encoding="utf-8-sig")
    df.to_excel("oliveyoung_reviews.xlsx", index=False)

    print(f"✅ 총 {len(all_reviews)}개의 리뷰 수집 완료")
    driver.quit()


if __name__ == "__main__":
    main()


In [ ]:
# ─────────────────────────────────────────
# 필요 패키지 (최초 1회만)
# pip install selenium webdriver-manager pandas openpyxl
# ─────────────────────────────────────────

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.remote.remote_connection import RemoteConnection
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    WebDriverException,
)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os

# ─── 0) HTTP 통신 타임아웃을 300초로 연장 ───
RemoteConnection.set_timeout(300)

# 저장할 CSV 파일 경로
CSV_PATH = "oliveyoung_reviews.csv"

def init_csv():
    """처음 실행 시 헤더가 달린 빈 CSV를 만들어 둡니다."""
    if not os.path.exists(CSV_PATH):
        df = pd.DataFrame(columns=["리뷰내용"])
        df.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")

def append_reviews_to_csv(reviews):
    """리스트 형태의 리뷰들을 CSV에 append하고, 저장 개수를 반환."""
    if not reviews:
        return 0
    df = pd.DataFrame(reviews, columns=["리뷰내용"])
    df.to_csv(CSV_PATH, mode="a", index=False, header=False, encoding="utf-8-sig")
    return len(reviews)

def go_to_next_page(driver, wait):
    try:
        curr_str = driver.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip()
        if not curr_str.isdigit():
            return False
        curr = int(curr_str)
    except (NoSuchElementException, WebDriverException):
        return False

    next_no = curr + 1

    # 같은 그룹 내 번호 클릭 시도
    try:
        btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]')))
        btn.click()
    except TimeoutException:
        # '다음 10페이지' 그룹 클릭 후 다시 시도
        try:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pageing > a.next"))).click()
            wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, f'div.pageing > a[data-page-no="{next_no}"]'))).click()
        except TimeoutException:
            return False

    # 실제 페이지 전환 확인
    try:
        wait.until(
            lambda d: d.find_element(By.CSS_SELECTOR, "div.pageing > strong").text.strip() == str(next_no)
        )
        return True
    except TimeoutException:
        return False

def collect_reviews(driver, wait):
    reviews = []
    # 리뷰 탭 열기
    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#reviewInfo > a"))).click()
    except TimeoutException:
        return reviews

    while True:
        try:
            # 짧게 5초만 대기
            short_wait = WebDriverWait(driver, 5, poll_frequency=0.5)
            short_wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#gdasList > li")))

            elems = driver.find_elements(
                By.CSS_SELECTOR,
                "#gdasList > li > div.review_cont > div.txt_inner"
            )
            for e in elems:
                txt = e.text.strip()
                if txt:
                    reviews.append(txt)

        except TimeoutException:
            break
        except WebDriverException:
            break

        if not go_to_next_page(driver, wait):
            break

    return reviews

def main():
    # CSV 초기화
    init_csv()

    # ─── 1) Chrome 옵션 세팅 ───────────────────
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "profile.managed_default_content_settings.images": 2
    })
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_experimental_option("detach", True)

    # ─── 2) 드라이버 및 타임아웃 설정 ───────────
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    driver.set_page_load_timeout(60)
    driver.set_script_timeout(30)
    wait = WebDriverWait(driver, 10, poll_frequency=0.5)

    # ─── 3) 메인 접속 ───────────────────────────
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")

    categories = ["스킨/토너", "에센스/세럼/앰플"]

    for cat in categories:
        # GNB 메뉴 열고 카테고리 클릭
        wait.until(EC.element_to_be_clickable((By.ID, "btnGnbOpen"))).click()
        wait.until(EC.element_to_be_clickable((By.LINK_TEXT, cat))).click()

        # 상품 리스트 로딩 대기
        wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "ul.cate_prd_list > li .prd_info a")
        ))
        main_handle = driver.current_window_handle

        # 상품 링크 수집
        hrefs = [
            a.get_attribute("href")
            for a in driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li .prd_info a")
        ]

        for idx, href in enumerate(hrefs, 1):
            try:
                driver.execute_script("window.open(arguments[0], '_blank');", href)
                driver.switch_to.window(driver.window_handles[-1])

                reviews = collect_reviews(driver, wait)
                saved = append_reviews_to_csv(reviews)
                print(f"✅ [{cat}] 상품 {idx}/{len(hrefs)}: 리뷰 {len(reviews)}개 수집, {saved}개 저장 완료")

            except Exception as e:
                print(f"❗️ 리뷰 수집 중 오류 ({cat} 상품 {idx}): {e}")

            finally:
                driver.close()
                driver.switch_to.window(main_handle)

    # ─── 4) Excel 저장 (마지막에 한번만) ─────────
    df_all = pd.read_csv(CSV_PATH, encoding="utf-8-sig")
    df_all.to_excel("oliveyoung_reviews.xlsx", index=False)
    print(f"📂 전체 리뷰 {len(df_all)}개 엑셀 저장 완료: oliveyoung_reviews.xlsx")

    driver.quit()

if __name__ == "__main__":
    main()
